# Dog, cat image classification problem set

* 이번 학습에서는 처음부터 끝까지 Dog, cat dataset에 대한 분류 model을 구현합니다

### [CUDA](http://pytorch.org/docs/stable/cuda.html)

* cuda를 이용하겠습니다


In [2]:
import torch
import numpy as np

# check if CUDA is available
train_on_gpu = torch.cuda.is_available()
#train_on_gpu = False

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    print('CUDA is available!  Training on GPU ...')

CUDA is available!  Training on GPU ...


---
## Load the [Data](http://pytorch.org/docs/stable/torchvision/datasets.html)

* 아미지를 다운로드 받습니다
* 폴더별로
 - test
 - train
 - validation

 data를 받습니다. 


In [3]:
!pip install googledrivedownloader

In [4]:
from os.path import exists
from google_drive_downloader import GoogleDriveDownloader as gdd
import tarfile 

#if exists("./Cat_Dog_data.tgz"):
#    !rm -rf ./Cat_Dog_data.tgz

gdd.download_file_from_google_drive(file_id='1WpY0qpe7yF5C5M52z1BMIzYVpDYiU3OV',
                                    dest_path = './Cat_Dog_data.tgz')

tf = tarfile.open("Cat_Dog_data.tgz")
tf.extractall()

## Problem 1 [20 points]: 

* Training, validation, test를 위한 dataloader, transform을 적절하게 준비해주세요
* 아래 data 준비하는 코딩을 수행하고, 아래 markdown에 준비한 과정 및 이유를 구체적으로 설명하세요
* 아래 답안 작성에 data의 구조에 대해서 설명하세요
* 코드에는 주석을 달아주세요.

In [0]:
# Coding
import torch
import numpy as np
from torchvision import datasets
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler


# training set 중에서 validation set으로 분할 할 비율 
valid_size = 0.2

#batch size = sample데이터 중 한번에 네트워크에 넘겨주는 데이터의 수
#만약 데이터가 1000개 있으면 20개씩 50개 그룹화(1 iteration = 20개의 데이터에 대해 학습)
#해서 1개 epoch 돔 (epoch = 3일때 ->전체 데이터를 3번에 걸처 학습)
#training이 잘 되지 않는다면 batch_size를 낮춰주면 된다.
batch_size = 32



#train data 는 많을수록 성능 좋아지기 때문에 같은 사진이더라도 변형해서 사용
train_transforms = transforms.Compose([
                                      #  transforms.RandomRotation(30),##랜덤하게 사진을 30도 돌림
                                      #  transforms.RandomResizedCrop(224),#랜덤하게 확대 또는 축수 해서 224로자름
                                      #  transforms.RandomHorizontalFlip(),#랜덤하게 flip(미러-> 거울에 비친거)하는 과정
                                       transforms.Resize(224),
                                       transforms.CenterCrop(224),#확대 축소
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

#transform = 사용자가 정의하는 여러가지 변환 parameter 
#test data 에는 변형하면 안됨 -> 크기 조정같은것만 가능 -> tsst는 학습하는 것이 아니기 때문에
test_transforms = transforms.Compose([transforms.Resize(224),
                                      transforms.CenterCrop(224),#확대 축소
                                      transforms.ToTensor(),#tensor로 바꾸는 작업
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])
#validation data 에는 변형하면 안됨 -> 크기 조정같은것만 가능 -> tsst는 학습하는 것이 아니기 때문에
validation_transforms = transforms.Compose([transforms.Resize(224),
                                      transforms.CenterCrop(224),#확대 축소
                                      transforms.ToTensor(),#tensor로 바꾸는 작업
                                      transforms.Normalize([0.485, 0.456, 0.406],
                                                           [0.229, 0.224, 0.225])])

# cat_dog_data/train 의 경로 사진들을 가져와서 train_data에 앞에서 정의한 train_transforms형태로 모두 넣어준다. 
train_data = datasets.ImageFolder('train', transform=train_transforms)

#cat_dog_data/test 의 경로 사진들을 가져와서 train_data에 앞에서 정의한 test_transforms형태로 모두 넣어준다. 
test_data = datasets.ImageFolder('test', transform=test_transforms)

validation_data = datasets.ImageFolder('validation', transform=validation_transforms)


#######################################################################################################
# # train data 에서 validation 빼기 위한 작업 -> 일정량을 빼서 validation으로 만들어줌#################
#######################################################################################################
# # validation 분할하기, 아래 코드 하나씩 찍어보시고 무엇을 하는지 확인해보세요
# num_train = len(train_data)#트레인 데이터 길이 
# print(num_train)

# indices = list(range(num_train))#길이들을 리스트 형식으로 바꿔줌 -> [1,2,3 ''' 20000]

# np.random.shuffle(indices)#리스트들을 랜덤하게 섞어줌 -> [2,6,1,2 '''5]

# #split을 int 형으로 계산해 주기 위해서 -> train data에서 0.2 비율만큼 랜덤으로 뺀 자료들을 int형으로 바꿔주는 과정
# #20000 * 0.2 = 4000
# split = int(np.floor(valid_size * num_train))# floor = 소수점을 내림해줌, int = int 형식으로 바꿔줌
# print(split)

# #랜덤으로 섞어서 가져온 데이터들을 나눠서 앞에껀 validation index 으로, 뒤에껀 train index으로 잡음
# train_idx, valid_idx = indices[split:], indices[:split] 


# #index들의 형태를 dateloader의 sample로 사용하기 위해서 변경해 준다. 
# train_sampler = SubsetRandomSampler(train_idx)
# valid_sampler = SubsetRandomSampler(valid_idx)

#######################################################################################################
# # train data 에서 validation 빼기 위한 작업 -> 일정량을 빼서 validation으로 만들어줌#################
#######################################################################################################



#64개의 그림을 한번에 받아서 trainloader에 저장한다. -> trainloader는 섞어서 가져온다
trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size,shuffle = True)
testloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)
validloader = torch.utils.data.DataLoader(validation_data, batch_size=batch_size)

#나중에 test 할때 사용할 class 정의 -> 고양이와 강아지 2개 이기 때문에
classes = ['cat','dog']

**분석 및 설명:**
1. 데이터의 양을 늘리기 위해 가져올때 회전,확대 및 축소, 사이즈 변경 을 해준다. -> 데이터 양 늘어나면 성능 좋아지기 때문에

2. 정규화인 nomalize를 통해 최적으로 될 수 있는 값들을 찾음 ->3개의 채널에 대해 평균mean과 분산var을 해주는 것 

3. 정규화 이유 =  데이터의 범위를 사용자가 원하는 범위로 제한하는 것 -> mean을 빼는 작업은 data를 0을 중심으로 두는 변형을 하고, std를 나누는 것은 값들을 -1에서 1사이로 좁히는 작업을 수행

4. imageFolder를 이용해 각각 train,test,validation 에 있는 사진들을 transforms의 형태로 가져옴

5. batch 사이즈로 그룹화해서 loader 에 저장 -> shuffle은 데이터 차별화 줘서 학습 잘되게 함



## Problem 2 [20 point]: Define the Network Architecture

* 구현하고자하는 network을 작성해주세요
* 아래 구현 방법과 이유를 구체적으로 설명하세요
* 코드에는 주석을 달아주세요. 
* 아래 모델을 구체적으로 설명하고, 설정 이유를 작성해주세요
* 본 과정에서는 직접 network을 구현하고, transfer learning은 활용하지 않도록 하겠습니다.

In [6]:
# 코드 작성
from torch import nn, optim
import torch.nn.functional as F

#네트워크에 값을 넣어서 model을 만든다. 네트워크의 init에는 파라미터(h와 W )가 자동으로 만들어지며 
#forward를 통해 relu와 softmax적용된 x가 나온다.
class Network(nn.Module):
    
    def __init__(self):#layer 들을 정의 하는 구간
        super(Network,self).__init__()
        
        #224*224*3의 이미지로 시작 -> 224 는 필터=5, padding=2 에서 크기 유지 
        self.conv1 = nn.Conv2d(3,16,5,padding=2)
        #앞에 3 = input채널 수 , 16 = output 채널 수 , 뒤에 5 = 필터 크기 , padding = zero pad로 edge를 0으로 채움.->
        #padding 쓰는 이유 = 결과 이미지크기가 줄어들지 않게 하기 위해서
        #따라서 3개의 채널을 5*5의 필터 16개 적용해서 conv2를 만든다. 
        
        #112*112*16
        self.conv2 = nn.Conv2d(16,32,5,padding=2)
        #앞에 채널의 크기를 맞춰줘야 함 -> conv1의 채널결과가 16이기 때문에 input채널을 16으로 맞춰줌
        #padding으로 이미지 크기 안줄게 하고 필터 수만 늘려준다. -> 필터수 많아지면 결과 좋아지기 때문에
        
        #56*56*32
        self.conv3 = nn.Conv2d(32,64,5,padding=2)
        
        #28*28*64
        self.conv4 = nn.Conv2d(64,128,5,padding=2)
        
        #14*14*128
        self.conv5 = nn.Conv2d(128,256,5,padding=2)
        
        self.pool = nn.MaxPool2d(2,2)
        #maxpool = 가장 큰 값(영향력이 큰것만 빼준다)을 뽑아 채널을 놔두고 사이즈만 줄이는 방법
        
        #maxpool 로 인해 작아짐
        self.fc1 = nn.Linear(256*7*7,6272)
        self.fc2 = nn.Linear(6272,3136)
        
        #최종 분류가 고양이와 강아지 2개 이기 때문에 output 2로 해줌
        self.fc3 = nn.Linear(3136,1568)
        self.fc4 = nn.Linear(1568,784)
        self.fc5 = nn.Linear(784,128)
        self.fc6 = nn.Linear(128,2)
        
        self.dropout = nn.Dropout(0.5)
        
    def forward(self, x):#layer들을 어떻게 쓸것인지 정하는 구간
        
        x = self.pool(F.relu(self.conv1(x)))#-> 결과값 = 112*112*16
        x = self.pool(F.relu(self.conv2(x)))#-> 결과값 = 56*56*32
        x = self.pool(F.relu(self.conv3(x)))#-> 결과값 = 28*28*64
        x = self.pool(F.relu(self.conv4(x)))#-> 결과값 = 14*14*128
        x = self.pool(F.relu(self.conv5(x)))#-> 결과값 = 7*7*256
        x = x.view(-1,256*7*7)#벡터형태로 바꿔줌

        x = self.dropout(x)
        # overfitting(trinning 데이터에 너무 학습이 되어서 test 데이터에서 결과가 작아지는 현상)
        # 을 막기위해몇개의 노드를 죽이고 남은 노드들을 통해서만 훈련을 하는 것
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = F.relu(self.fc4(x))
        x = self.dropout(x)
        x = F.relu(self.fc5(x))
        x = self.dropout(x)
        x = F.log_softmax(self.fc6(x),dim=1)
        # log_softmax = 확률을 분포로 나타내기 위해서
        
        return x

print('완료')


완료


**분석 및 설명:** 
1. cnn -> image를 input으로 받을 때 유용한 네트워크로 더 복잡한 특징 있을 때 사용

2. convolutional -> 우리 뇌와 비슷한 형태로 동작하게 함 -> 여러개의 필터 적용하여 어떤 필터 사용했을 때 최대로 활성화 되는지 찾아준다.

3. 필터 갯수는 input필터(앞에필터) 갯수와 같아야 하며 필터 적용할 때마다 relu 함수와 maxpool 적용시켜서 학습시킨다.

4. 필터 크기 5와 padding 크기 2를 이용해 이미지 크기 달라지지 않게 하여 채널 수만 증가시킨다. -> maxpool(2,2)로 부분별 이미지 픽셀 수를 줄여 영향력이 큰 값만 추출한다. -> 채널수는 그대로고 파라미터 수만 줄인다. 

5. dropout을 이용해 overfitting 을 방지해 학습률을 높인다. -> 너무 trainung data에 적합하게 학습되면 안됨

6. relu -> 0 이하 영역의 기울기를 0으로 만든 active 함수로 학습이 잘되게 해준다.-> gradient (기울기가 0과 비슷한 부분 = 포화영역)이 많으면 학습이 잘 되지 않기 때문에 ->sigmoid는 포화영역이 2두군데 

7. dropout = 중간에 랜덤하게 activation 들을 끊어 성능을 높임 -> 각각의 중요도를 분산시켜 어느 한 가지 특징만 중요하지 않게 만든다



## Problem 3 [5 point]: Specify Loss Function

* Loss 함수와 optimizer를 구현하세요
* 선정 이유를 설명하세요


In [13]:
# 코드

model = Network()
print(model)
#만약 device가 cuda를 이용 가능하면 cuda로 하고 그렇지 않다면 cpu로 해라
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if train_on_gpu:
      model.cuda()


criterion = nn.NLLLoss()#cross antropy 말고 NLLLoss()쓴다.
optimizer = optim.Adam(model.parameters(), lr=0.001,weight_decay=1e-7)#무엇을 최적화 할 것인지 써주는것
#adam = momentum 과 RMSProp 합친거
#optimizer 종류 = adam,SGD, SGD-Momentum 등  -> gradient(기울기)로 

dataiter = iter(trainloader)
images, labels = dataiter.next()
print(type(images))
print(images.shape)
print(labels.shape)


Network(
  (conv1): Conv2d(3, 16, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(16, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv3): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv4): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv5): Conv2d(128, 256, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=12544, out_features=6272, bias=True)
  (fc2): Linear(in_features=6272, out_features=3136, bias=True)
  (fc3): Linear(in_features=3136, out_features=1568, bias=True)
  (fc4): Linear(in_features=1568, out_features=784, bias=True)
  (fc5): Linear(in_features=784, out_features=128, bias=True)
  (fc6): Linear(in_features=128, out_features=2, bias=True)
  (dropout): Dropout(p=0.5, inplace=False)
)
<class 'torch.Tensor'>
torch.Size([32, 3, 224, 224])
torch.Size([32])


 **설명:**
 1. nn.NLLLoss를 사용한다. => cross 는 softmax안취해 주기 때문에
 
 2. optimizer -> 학습률을 줄여나가고 속도를 계산하여 학습을 조정해나가는 방법 ->  방향과 학습률 두 가지를 모두 잡기 위한 방법
 
 3. lr = 학습비율(learning rate)로 최적점에서 멀리 있을 때는 lr을 크게 하고, 점차 최소점에 나아갈수록 lr을 작게 만드는 것이 좋음



## Problem 4 [30 point]: Train the Network

* training loss와 validation loss를 기록하며 training을 구현하세요
* 만약 validation loss가 최소화된 모델을 저장하세요
* 코드에는 모두 주석을 포함해주세요
* training과정을 설명하고, training 결과를 분석해주세요

In [12]:
# 코드 작성

#model의 결과를 device 즉 cuda로 보내서 연산할 거다. 
#model.to(device)

#batch size = sample데이터 중 한번에 네트워크에 넘겨주는 데이터의 수 -> 그룹화 됨
#epoch 줄여서 overfiting 막을 수 있음
epochs = 5
a = 0
validation_losses_min = np.Inf # 무한대를 표현하기 위해서 np.inf사용


for e in range(epochs):

    
    train_losses = 0
    validation_losses = 0
    a = a + 1
    print(a)
    
    model.train()#켜주고
    for idx, (images, labels) in enumerate(trainloader):
        if idx == 2:
          break
        
        #앞의 model.to와 같이 input값과 label 값들도 gpu로 계산하기 위해서 cuda형식으로 바꿔서 계산
        #GPU를 쓰기위해 data 와 target을 cuda형식으로 바꿔줌
        if train_on_gpu:
             images, labels = images.cuda(), labels.cuda()

        optimizer.zero_grad()#여기서 다시 grad 켜준다.
        log_ps = model(images)
        loss = criterion(log_ps, labels)#앞 네트워크에서 작업한 self와 x 의 인수로 들어감
        loss.backward()
        
        #update하는 과정, 파라미터 바꿔줌
        optimizer.step()

        train_losses += loss.item()*images.size(0)

        #data.size 곱해주는 이유 = loss.item은 전체 데이터 셋을 나눈 값이기 때문에 나중에 평균 내려면
        #20으로 나누기 전 값이 train_loss에 각각 더해저 나중에 dataset의 크기로 나눠 평균을 구할 수 있다.
    
    #validation 값들을 출력해줄 변수
    accuracy = 0

    # Gradient를 계산할 필요 없으니 비활성화
    # with torch.no_grad():
    ##validation 과정에서는 dropout을 비활성화 ->droupout 확률이 0으로 동작


    model.eval()#꺼주고->#validation 학습 안하게 함

    #traing 다 끝나고 돌린다. validation 하는 부분
    for images, labels in validloader:
      
      if train_on_gpu:
        images, labels = images.cuda(), labels.cuda()
      
                
      log_ps = model(images)
      loss = criterion(log_ps, labels)
      validation_losses += loss.item()*images.size(0)

      ps = torch.exp(log_ps)#log값 풀어주기 위해서 
      top_p, top_class = ps.topk(1, dim=1)#각각 row에 대해 제일 큰값의 인덱스를 리턴해준다.
                
      #eqals에다가 비교한값을 넣어준다. 이때 label의 형태를 top_class와 맞춰준다.
      #equals에는 각 batch의 성공과 실패를 담고있다. 
      #top_class와 label를 비교 = 전체 데이터에 대한 성공률을 계산 -> validation 값이 나옴
      equals = top_class == labels.view(*top_class.shape)
                
      #equals의 타입을 float로 바꿔주고 torch.mean으로 평균을 내주면 해당 batch의 성공률이 나온다.
      accuracy += torch.mean(equals.type(torch.FloatTensor))

    
    train_losses = train_losses/len(trainloader)
    validation_losses = validation_losses/len(validloader)

    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(e, train_losses, validation_losses))
    if validation_losses <= validation_losses_min:
        #이전단계보다 잘 학습이 됬을 경우 저장해라.
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(validation_losses_min, validation_losses))
        torch.save(model.state_dict(), 'model_cifar.pt')#model_cifar 의 이름으로 저장해라
        
        #이전단계와 계속 비교해야 하니까 최소값을 저장된 validation(더낮은)것으로 저장해줌
        validation_losses_min = validation_losses
    


#     print("Epoch: {}/{}.. ".format(e+1, epochs),
#           "Training Loss: {:.3f}.. ".format(train_losses[-1]),
#           "validation Loss: {:.3f}.. ".format(validation_losses[-1]),
#           "Test Accuracy: {:.3f}".format(accuracy/len(testloader)))

1
Epoch: 0 	Training Loss: 0.071302 	Validation Loss: 21.958524
Validation loss decreased (inf --> 21.958524).  Saving model ...
2
Epoch: 1 	Training Loss: 0.095086 	Validation Loss: 21.943929
Validation loss decreased (21.958524 --> 21.943929).  Saving model ...
3
Epoch: 2 	Training Loss: 0.072014 	Validation Loss: 21.965651
4
Epoch: 3 	Training Loss: 0.071076 	Validation Loss: 21.963665
5
Epoch: 4 	Training Loss: 0.070924 	Validation Loss: 21.966271


**분석 및 설명:**
1. validation set(trining, test) 와 같이 분류됨 -> 총 3가지 data set으로 분류됨

2. validation set 사용 이유1 = 모델의 성능을 평가하기 위해서 -> overfitting 을 예측할 수 있다.validation set 사용 이유2 = 언제 학습을 멈출지를 판단하기 위해 이용 -> trining 과정에 관여하며 training이 된 여러가지 모델 중 가장 좋은 하나의 모델을 고르기 위한 셋 -> trining set에서 추출된 dataset이다.

3. eqoch을 높여 학습량을 늘려준다. -> 학습률 높아짐

4. model.eval(),model.train() -> dropout 활성/비활성 하는 것 -> validation 과정에서는 dropout을 비활성화 -> droupout 확률이 0으로 동작-> training에서만 활성화 하고 test할때는 꺼줌으로 써 학습되지 않게 만들어줌

5. gpu로 연산하게 하여 연산 속도를 높여줌

6. backword -> #gradient 구하기 위해 사용 ->backpropagation 과정이 함축되 있음. -> 계속 기울기 줄여서 최소되는 기울기 찾음 ->  최적화 하기 위해서 -> 더 자세한 과정 = 뒤에서 앞으로 계산해 나가는 과정으로, (앞단계에서 흘러들어온 미분 값) * (로컬 미분값) 한 것 -> 대개 forward pass 계산될 때 backward 도 함께 수행되서 저장된다.

7. state_dict = 트레이닝완료된 파라미터 값들을 key와 value형태로 가지고 있는 것



## Problem 5 [5 point] Validation Loss가 최소화된 Model가져오기

* 최소 validation loss를 갖는 model을 불러옵니다

In [0]:
# 코드 작성
#위에서 사용한 parameter중 가장 최적화 된 parameter 가 저장되었으니 그걸 불러온다.
model.load_state_dict(torch.load('model_cifar.pt'))

<All keys matched successfully>

---
## Problem 6 [20point]: Test the Trained Network

* Test set을 활용하여 성능 검증
* Accuracy (분류 성공률)와 test loss를 출력하세요
* 코드에는 주석을 달아주세요
* 아래 test 결과에 대해서 간단하게 설명/분석 해주세요

In [26]:

test_loss = 0.0
class_correct = list(0. for i in range(2))#2개의 0으로 된 list 만듬
class_total = list(0. for i in range(2))

model.eval()#test 여서 학습 안하게 함

for images, labels in testloader:

    if train_on_gpu:
        images, labels = images.cuda(), labels.cuda()
        

    output = model(images)
    loss = criterion(output, labels)
    test_loss += loss.item()*images.size(0)
    
    #이중에서 확률이 가장 높은 것을 pred에다가 넣어줘라. ->1차원으로 봐야함
    _, pred = torch.max(output, 1)
    
    #pred와 같은지 비교하는 부분 -> labels date의 형태를 pred 와 같이 변경시켜줘라
    correct_tensor = pred.eq(labels.data.view_as(pred))
    
    #만약 GPU가 아니면 그냥 numpy 해주고 GPU에서 사용했으면 cpu로 넘겨줘서 numpy 해줘라
    #numpy 연산은 gpu에서 안되기 때문에 cpu로 넘겨주는 과정
    correct = np.squeeze(correct_tensor.numpy()) if not train_on_gpu else np.squeeze(correct_tensor.cpu().numpy())
    
    #몇개 맞췄는지 개산 
    for i in range(len(labels.data)):#batch size = sample데이터 중 한번에 네트워크에 넘겨주는 데이터의 수
        label = labels.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1


test_loss = test_loss/len(testloader.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(2):
    if class_total[i] > 0:
        #몇개 틀리고 맞았는지 알려주는 과정
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))

Test Loss: 0.693156

Test Accuracy of   cat:  0% ( 0/1250)
Test Accuracy of   dog: 100% (1250/1250)

Test Accuracy (Overall): 50% (1250/2500)


In [25]:
images.shape
labels.shape
labels.data.shape
len(labels.data)


32

**분석 및 설명:**
1. 학습된 것을 바탕으로 test를 하는 과정

2. test에 있는 새로운 이미지들을 이용해 얼마나 학습이 잘 됬는지 확인한다.

3. 결과값 output이 2(cat,dog)개이기 때문에 2번만 돌려서 학습률을 확인한다.


---
## Problem 7: 전체적인 총평

* Data준비, Training과 validation 과정에서 성능 개선을 위해서 수행한 과정과 성공/실패 이유를 분석해주세요


**설명:**
1. image 학습과 복잡한 학습에 좋은 cnn를 사용하여 성공률을 높였다.
2. relu함수를 적용시켜 학습이 잘되도록 했다.
3. maxpool을 사용하여 가장 학습 잘된 값들을 찾았다.
4. nomalize를 통해 정규화를 시켜줬다.
